In [2]:
import numpy as np
import pandas as pd
import os
import numpy as np
from csv import writer

In [73]:
subreddits = [] #contains list of created subreddit.csv files

path_ndjson = "E:\\School\\U-of-E-Schoolwork\\MLP\\data\\ndjson\\reddit.ndjson"
path_data = "E:\\School\\U-of-E-Schoolwork\\MLP\\data\\csv" #path to folder to contain the csv files
cols = ['author', 'date_post', 'id_post', 'number_post', 'subreddit', 'text_post', 'thread']

def append_csv(values:list) -> None:
    with open(os.path.join(path_data, f"{values[4]}.csv"), 'a', encoding='utf-8', newline='') as file:
        writer_obj = writer(file)
        writer_obj.writerow(values)

        file.close()

def write_csv(values:list) -> None:
    subreddits.append(values[4])

    df = pd.DataFrame(data=[values], columns=cols)
    df.to_csv(os.path.join(path_data, f"{values[4]}.csv"), index=None)

def fun_csv(series : list) -> None:
    sr = series[4]
    if sr in subreddits:
        append_csv(series)
    else:
        write_csv(series)

with pd.read_json(
        path_or_buf=path_ndjson,
        lines=True,
        chunksize=20) as reader:
    idx = 1
    #cycle through the iterator
    for chunk in reader:
        idx += 1
        if idx > 20:
            break
        chunk.apply(fun_csv, axis = 1, raw=True)


In [42]:
def fun_csv(series : pd.Series) -> None:
    print(series)
    return

df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
df.apply(fun_csv, axis=0, raw=True)

[4 4 4]
[9 9 9]


A    None
B    None
dtype: object

In [75]:
path_ndjson = "E:\\School\\U-of-E-Schoolwork\\MLP\\data\\ndjson\\reddit.ndjson"
path_data = "E:\\School\\U-of-E-Schoolwork\\MLP\\data\\csv" #path to folder to contain the csv files
def fun_csv(series : list) -> None:
    sr = series[4]
    #print(series)
with open("E:\\School\\U-of-E-Schoolwork\\MLP\\data\\ndjson\\reddit.ndjson", 'r') as reader:
    idx = 0
    #cycle through the iterator
    for chunk in reader:
        idx += 1

print(idx)

29482387


In [3]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [4]:
%%cython

import os
import cython
import numpy as np
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer
import argparse
import string
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
import json
import re
import pandas as pd
import time

nltk.download('stopwords')
nltk.download('punkt')
string.punctuation

parser = argparse.ArgumentParser()
parser.add_argument('--data_path', required=True)
parser.add_argument('--output_dir', required=True)
parser.add_argument('--name', required=True)
parser.add_argument('--min_df', type=int, default=20)
parser.add_argument('--chunksize', type=int, default=50)
args = parser.parse_args()

cdef list texts = []
cdef list stopwords_list = stopwords.words('english')

cdef void process(text):
    # remove websites
    input_str = re.sub(r'http\S+', '', text)


    # lower caps
    input_str = input_str.lower()

    # remove punctuation
    input_str = input_str.translate(
        str.maketrans("", "", string.punctuation)
    )

    # remove trailing spaces
    input_str = input_str.strip()

    # tokenize
    input_str = wordpunct_tokenize(input_str)

    # remove stop words
    input_str_w_sw = [word for word in input_str if not word in stopwords_list]

    # filter out short texts
    if len(input_str_w_sw) < args.min_df:
        pass
    else:
        input_str = " ".join(input_str_w_sw)
        texts.append(input_str)

cdef void run():
    with pd.read_csv(args.data_path, chunksize=args.chunksize) as reader:
        for chunk in tqdm(reader):
            chunk.apply(lambda x: process(x['text_post']), axis=1, raw=False)

    print('vectorizing')

    vectorizer = CountVectorizer(min_df=args.min_df)
    bow_matrix = vectorizer.fit_transform(texts).toarray()

    idx = np.where(bow_matrix.sum(axis=-1) > 0)
    bow_matrix = bow_matrix[idx]

    vocab = vectorizer.get_feature_names_out()

    print(vocab)

    print("===>saving files")

    os.makedirs(os.path.join(args.output_dir, f'{args.name}'), exist_ok=True)

    scipy.sparse.save_npz(os.path.join(args.output_dir, f'{args.name}', f'bow_matrix.npz'),
                          scipy.sparse.csr_matrix(bow_matrix))
    with open(os.path.join(args.output_dir, f'{args.name}', f'vocab.txt'), 'w') as file:
        for line in vocab:
            file.write(line + '\n')

    print('===>done.')

DistutilsPlatformError: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/